# CropHarvest Demo

In [1]:
from cropharvest.datasets import CropHarvestLabels

In [2]:
labels = CropHarvestLabels("/Users/izvonkov/nasaharvest/test", download=False)

In [3]:
len(labels)

90528

In [4]:
labels[0]

index                                                                   0
is_crop                                                                 1
lat                                                               10.7126
lon                                                               37.0831
dataset                                                          ethiopia
collection_date                                       2020-10-22T00:00:00
export_end_date                                       2021-02-01T00:00:00
harvest_date                                                         None
planting_date                                                        None
label                                                                None
classification_label                                                 None
geometry                POLYGON ((37.0825205254332 10.712735028397, 37...
Name: 0, dtype: object